模块化的双通道注意力

In [1]:
import os
import sys
import torch


# 导入自定义模块
from config import DEVICE, MODEL_PATH
from data_loader import verify_data_loading, create_data_loaders
from debug_tools import ModelDebugger, test_basic_functionality
from model_components import ImprovedDualPathModel
from train_utils import train_model, test_model,train_model_with_pretrain,finetune_resnet50
from visualization import visualize_training_results

In [2]:
def setup_environment():
    """设置环境"""
    print("=" * 60)
    print("🌱 双路径细粒度花卉分类系统")
    print("=" * 60)

    print(f"设备: {DEVICE}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"内存: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

    # 清理缓存
    torch.cuda.empty_cache()

    return DEVICE

In [3]:
def debug_mode(device):
    """调试模式"""
    print("\n" + "=" * 60)
    print("🔍 调试模式")
    print("=" * 60)

    # 1. 测试基础功能
    test_basic_functionality(device)

    # 2. 验证数据加载
    print("\n" + "=" * 60)
    print("📊 验证数据加载")
    print("=" * 60)
    verify_data_loading()

    # 3. 测试模型组件
    print("\n" + "=" * 60)
    print("🤖 测试模型组件")
    print("=" * 60)

    model = ImprovedDualPathModel(num_classes=5, num_regions=2).to(device)
    debugger = ModelDebugger()

    # 测试前向传播
    outputs = debugger.test_forward_pass(model, device=device)

    # 测试梯度
    print(f"\n📈 测试模型梯度:")
    model.train()
    dummy_input = torch.randn(2, 3, 224, 224).to(device)
    dummy_labels = torch.randint(0, 5, (2,)).to(device)

    outputs = model(dummy_input)
    criterion = torch.nn.CrossEntropyLoss()
    loss, _ = model.compute_loss(outputs, dummy_labels, criterion)
    loss.backward()

    debugger.check_gradient(model, loss)

    return model


In [4]:
def train_mode(device):
    """训练模式"""
    print("\n" + "=" * 60)
    print("🚀 训练模式")
    print("=" * 60)

    # 训练模型
    model, train_losses, train_accs, test_losses, test_accs = train_model(device)

    # 可视化结果
    visualize_training_results(train_losses, train_accs, test_losses, test_accs)

    return model


In [5]:
def test_mode(device):
    """测试模式"""
    print("\n" + "=" * 60)
    print("🧪 测试模式")
    print("=" * 60)

    # 测试模型
    test_acc = test_model(device)

    # 可视化注意力（如果有测试数据）
    try:
        from data_loader import create_data_loaders
        _, test_loader, _, _ = create_data_loaders(batch_size=4)

        model = ImprovedDualPathModel().to(device)
        checkpoint = torch.load(MODEL_PATH, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])

        from visualization import visualize_sample_attention
        visualize_sample_attention(model, test_loader, device, num_samples=2)

    except Exception as e:
        print(f"⚠️  无法可视化: {e}")

    return test_acc

In [6]:
def main():
    # 修复Jupyter的__file__问题
    if '__file__' not in globals():
        notebook_dir = os.getcwd()  # 使用当前目录
        globals()['__file__'] = os.path.join(notebook_dir, 'main.py')
        sys.path.insert(0, notebook_dir)

    # 设置环境
    device = setup_environment()

    # Jupyter中改为手动选择模式
    print("\n🎮 请选择运行模式:")
    print("1. 调试模式 (debug)")
    print("2. 训练模式 (train)")
    print("3. 测试模式 (test)")
    print("4. 完整流程 (all)")

    try:
        choice = input("请输入数字 (1/2/3/4): ").strip()
    except:
        choice = '1'  # 默认选择调试模式

    # 根据选择运行
    if choice == '1':
        debug_mode(device)
    elif choice == '2':
        freeze_choice = input("请选择 (a/b): ").strip().lower()
        freeze_backbone = (freeze_choice == 'a')

        from train_utils import finetune_resnet50
        finetune_resnet50(device, freeze_backbone=freeze_backbone)
    elif choice == '3':
        test_mode(device)
    elif choice == '4':
        print("\n" + "=" * 60)
        print("🔄 完整流程")
        print("=" * 60)

        # 调试
        model = debug_mode(device)

        # 训练
        if input("\n是否继续训练? (y/n): ").lower() == 'y':
            train_mode(device)

        # 测试
        if input("\n是否测试模型? (y/n): ").lower() == 'y':
            test_mode(device)

    print("\n" + "=" * 60)
    print("✅ 程序执行完成")
    print("=" * 60)

In [ ]:
if __name__ == '__main__':
    main()

🌱 双路径细粒度花卉分类系统
设备: cuda
GPU: NVIDIA GeForce RTX 4060 Laptop GPU
内存: 8.18 GB

🎮 请选择运行模式:
1. 调试模式 (debug)
2. 训练模式 (train)
3. 测试模式 (test)
4. 完整流程 (all)

🚀 开始微调ResNet50模型...
🔧 创建完全匹配的ResNet50模型...
🔧 修改全连接层以匹配预训练模型...
📥 加载预训练权重...
✅ 权重加载成功！完全匹配

🎯 模型验证:
骨干网络: ResNet50
特征维度: 2048
分类器结构: 2048 → 1024 → 512 → 102
测试输入: torch.Size([2, 3, 224, 224])
测试输出: torch.Size([2, 102])
输出维度: 102 (应该是102)

🔒 冻结骨干网络...
  🔓 训练: fc.0.weight
  🔓 训练: fc.0.bias
  🔓 训练: fc.3.weight
  🔓 训练: fc.3.bias
  🔓 训练: fc.6.weight
  🔓 训练: fc.6.bias

📊 冻结统计:
冻结参数: 159 个层
可训练参数: 6 个层

📊 数据统计:
训练集: 1020 样本
测试集: 6149 样本

📈 开始训练，共 20 个epoch...
  Batch 0/63: Loss: 0.0369, Acc: 1.0000
  Batch 10/63: Loss: 0.2563, Acc: 0.8239
  Batch 20/63: Loss: 0.1952, Acc: 0.8512
  Batch 30/63: Loss: 0.0777, Acc: 0.8790
  Batch 40/63: Loss: 0.1056, Acc: 0.8902
  Batch 50/63: Loss: 0.0190, Acc: 0.9069
  Batch 60/63: Loss: 0.0597, Acc: 0.9139

📊 Epoch 1/20:
  训练损失: 0.2826, 训练准确率: 0.9167
  测试损失: 0.6582, 测试准确率: 0.8588
  学习率: 0.000976
  💾 保存最佳模型，准确率: 0

In [8]:
NUM_CLASSES = 102
def check_model_type():
    """检查模型类型"""
    print("🔍 检查 model_89.pth 类型...")

    try:
        checkpoint = torch.load("model_89.pth", map_location='cpu')

        # 提取state_dict
        if isinstance(checkpoint, dict):
            if 'model_state_dict' in checkpoint:
                state_dict = checkpoint['model_state_dict']
                print(f"📦 包含model_state_dict，有{len(state_dict)}个参数")
            elif 'state_dict' in checkpoint:
                state_dict = checkpoint['state_dict']
                print(f"📦 包含state_dict，有{len(state_dict)}个参数")
            else:
                state_dict = checkpoint
                print(f"📦 直接是state_dict，有{len(state_dict)}个参数")
        else:
            state_dict = checkpoint
            print(f"📦 直接是state_dict，有{len(state_dict)}个参数")

        # 获取所有键
        keys = list(state_dict.keys())
        print(f"\n📋 前10个参数键:")
        for i, key in enumerate(keys[:10]):
            print(f"  {i+1:2d}. {key}: {state_dict[key].shape}")

        # 判断模型类型
        if 'fc.weight' in keys and 'fc.bias' in keys:
            print(f"\n✅ 这是标准ResNet模型")
            print(f"  全连接层: fc.weight {state_dict['fc.weight'].shape}")
            print(f"  全连接层: fc.bias {state_dict['fc.bias'].shape}")
            return 'resnet'

        # 检查双路径模型特征
        dualpath_keys = ['global_classifier', 'region_proposal', 'adaptive_fusion',
                        'final_classifier', 'region_classifiers']
        found_dualpath = [key for key in dualpath_keys if any(key in k for k in keys)]

        if found_dualpath:
            print(f"\n✅ 这是双路径模型")
            print(f"  包含的双路径组件: {found_dualpath[:3]}")
            return 'dualpath'

        # 检查是否有骨干网络层
        resnet_layers = ['layer1', 'layer2', 'layer3', 'layer4', 'conv1', 'bn1']
        found_resnet = [layer for layer in resnet_layers if any(layer in k for k in keys)]

        if found_resnet:
            print(f"\n🔍 包含ResNet骨干层: {found_resnet}")
            # 检查输出维度
            for key in keys:
                if 'weight' in key and ('fc' in key or 'classifier' in key):
                    if state_dict[key].dim() == 2:  # 线性层权重
                        out_features = state_dict[key].shape[0]
                        print(f"  分类器输出维度: {out_features}")
                        if out_features == NUM_CLASSES:
                            print(f"  匹配当前类别数: {NUM_CLASSES}")
                        return 'custom_resnet'

        # 检查是否是简单的分类模型
        classifier_keys = [k for k in keys if 'classifier' in k or 'fc' in k or 'linear' in k]
        if classifier_keys:
            print(f"\n🔍 找到分类器层: {classifier_keys}")
            for key in classifier_keys[:2]:
                print(f"  {key}: {state_dict[key].shape}")
            return 'custom_classifier'

        print(f"\n⚠️  未知模型结构")
        print(f"  总参数数: {len(keys)}")

        # 保存结构信息到文件
        with open('model_structure.txt', 'w') as f:
            f.write("模型结构分析:\n")
            f.write("="*50 + "\n")
            for key in keys:
                f.write(f"{key}: {state_dict[key].shape}\n")

        print(f"📝 详细结构已保存到 model_structure.txt")
        return 'unknown'

    except Exception as e:
        print(f"❌ 检查失败: {e}")
        return 'error'

In [9]:
#check_model_type()

🔍 检查 model_89.pth 类型...
📦 直接是state_dict，有324个参数

📋 前10个参数键:
   1. conv1.weight: torch.Size([64, 3, 7, 7])
   2. bn1.weight: torch.Size([64])
   3. bn1.bias: torch.Size([64])
   4. bn1.running_mean: torch.Size([64])
   5. bn1.running_var: torch.Size([64])
   6. bn1.num_batches_tracked: torch.Size([])
   7. layer1.0.conv1.weight: torch.Size([64, 64, 1, 1])
   8. layer1.0.bn1.weight: torch.Size([64])
   9. layer1.0.bn1.bias: torch.Size([64])
  10. layer1.0.bn1.running_mean: torch.Size([64])

🔍 包含ResNet骨干层: ['layer1', 'layer2', 'layer3', 'layer4', 'conv1', 'bn1']
  分类器输出维度: 1024


'custom_resnet'

In [10]:
def analyze_model_structure():
    """详细分析模型结构"""
    print("🔍 详细分析 model_89.pth 结构...")

    checkpoint = torch.load("model_89.pth", map_location='cpu')

    # 找出所有线性层/分类器层
    linear_layers = {}
    conv_layers = {}
    bn_layers = {}

    for key, value in checkpoint.items():
        if 'weight' in key and value.dim() == 2:  # 线性层
            linear_layers[key] = value.shape
        elif 'weight' in key and value.dim() == 4:  # 卷积层
            conv_layers[key] = value.shape
        elif 'weight' in key and value.dim() == 1:  # BN层权重
            bn_layers[key] = value.shape

    print(f"\n📊 层类型统计:")
    print(f"线性层: {len(linear_layers)} 个")
    print(f"卷积层: {len(conv_layers)} 个")
    print(f"BN层权重: {len(bn_layers)} 个")

    print(f"\n🎯 线性层（分类器）:")
    for key, shape in linear_layers.items():
        print(f"  {key}: {shape}  (输入: {shape[1]}, 输出: {shape[0]})")

    print(f"\n🔍 最后一层分析:")
    # 查找可能的分类器层
    classifier_keys = [k for k in checkpoint.keys()
                      if 'fc' in k or 'classifier' in k or 'linear' in k]
    if classifier_keys:
        for key in classifier_keys:
            if 'weight' in key:
                shape = checkpoint[key].shape
                print(f"  候选分类器: {key}")
                print(f"    权重形状: {shape}")
                print(f"    输出类别数: {shape[0]}")

    # 如果没有明确的分类器，找最后一个线性层
    if not classifier_keys and linear_layers:
        last_key = list(linear_layers.keys())[-1]
        shape = linear_layers[last_key]
        print(f"\n📌 最后一个线性层可能是分类器:")
        print(f"  {last_key}: {shape}")
        print(f"  输出维度: {shape[0]} (可能是类别数)")

    # 检查是否是layer4的输出特征数
    print(f"\n🔧 骨干网络特征维度:")
    # 查找layer4的输出通道
    for key in conv_layers.keys():
        if 'layer4' in key and 'conv' in key:
            print(f"  {key}: {conv_layers[key][0]} 个输出通道")

    return checkpoint

In [11]:
analyze_model_structure()

🔍 详细分析 model_89.pth 结构...

📊 层类型统计:
线性层: 3 个
卷积层: 53 个
BN层权重: 53 个

🎯 线性层（分类器）:
  fc.0.weight: torch.Size([1024, 2048])  (输入: 2048, 输出: 1024)
  fc.3.weight: torch.Size([512, 1024])  (输入: 1024, 输出: 512)
  fc.6.weight: torch.Size([102, 512])  (输入: 512, 输出: 102)

🔍 最后一层分析:
  候选分类器: fc.0.weight
    权重形状: torch.Size([1024, 2048])
    输出类别数: 1024
  候选分类器: fc.3.weight
    权重形状: torch.Size([512, 1024])
    输出类别数: 512
  候选分类器: fc.6.weight
    权重形状: torch.Size([102, 512])
    输出类别数: 102

🔧 骨干网络特征维度:
  layer4.0.conv1.weight: 512 个输出通道
  layer4.0.conv2.weight: 512 个输出通道
  layer4.0.conv3.weight: 2048 个输出通道
  layer4.1.conv1.weight: 512 个输出通道
  layer4.1.conv2.weight: 512 个输出通道
  layer4.1.conv3.weight: 2048 个输出通道
  layer4.2.conv1.weight: 512 个输出通道
  layer4.2.conv2.weight: 512 个输出通道
  layer4.2.conv3.weight: 2048 个输出通道


OrderedDict([('conv1.weight',
              tensor([[[[ 1.3335e-02,  1.4664e-02, -1.5351e-02,  ..., -4.0896e-02,
                         -4.3034e-02, -7.0755e-02],
                        [ 4.1205e-03,  5.8477e-03,  1.4948e-02,  ...,  2.2060e-03,
                         -2.0912e-02, -3.8517e-02],
                        [ 2.2331e-02,  2.3595e-02,  1.6120e-02,  ...,  1.0281e-01,
                          6.2641e-02,  5.1977e-02],
                        ...,
                        [-9.0349e-04,  2.7767e-02, -1.0105e-02,  ..., -1.2722e-01,
                         -7.6604e-02,  7.8453e-03],
                        [ 3.5894e-03,  4.8006e-02,  6.2051e-02,  ...,  2.4267e-02,
                         -3.3662e-02, -1.5709e-02],
                        [-8.0029e-02, -3.2238e-02, -1.7808e-02,  ...,  3.5359e-02,
                          2.2439e-02,  1.7077e-03]],
              
                       [[-1.8452e-02,  1.1415e-02,  2.3850e-02,  ...,  5.3736e-02,
                          4.4022